1. Import the dataset, visualise some images.
2. Print one ProductId's image, attributes and description.

In [64]:
import numpy as np
location = 'attributes.csv'
tmp = np.genfromtxt(location, delimiter=',', dtype=np.string_) #load csv
attributes = tmp[1:,:] #remove headers
attributes_headers = tmp[0]
print(attributes_headers)
print(attributes.shape)

['ProductId' 'AttributeName' 'AttributeValueName']
(13455, 3)


In [59]:
location = 'products.csv'
tmp = np.genfromtxt(location, delimiter=',', dtype=np.string_, comments=None, usecols=np.arange(0,12)) #added 2 conditions for import to work 
products = tmp[1:,:] 
products_headers = tmp[0]
print(products_headers)
print(products.shape)

['ProductId' 'Description' 'DescriptionDate' 'SeasonOriginal'
 'ProductName' 'Brand' 'Gender' 'Colour' 'Family' 'Category'
 'ArticlePhotoId' 'CreateDate']
(12631, 12)


In [60]:
# Images are too heavy for import, get them when needed
# First let's see how many images we have for all products 0,1,2 ?
# Need to import the names of all images while stripping the jpg header


In [63]:
print(products[545])
print(attributes[545])

['11326917'
 'White and blue striped 3/4 sleeve top from Pleats Please By Issey Miyake. '
 '2016-02-16 22:38:11.413' 'SS16' 'striped 3/4 sleeve top'
 'PLEATS PLEASE BY ISSEY MIYAKE' 'WOMEN' 'BLUE' 'Clothing' 'Tops'
 '6300788' '2016-01-28 14:00:31.883']
['11314649' 'Sleeve Length' 'Shortsleeved']


In [91]:
import os
imageFullNames = os.listdir('images')
imageIds = [a.split("_", 1)[0] for a in imageFullNames] #remove file name after ProductId
print(imageFullNames[56])
print(imageIds[56])

11509675_7188420_255.jpg
11509675
